In [2]:
import os
import sys
import pandas as pd
import json
import csv

In [53]:
reload(sys)
sys.setdefaultencoding("utf-8")
f_train = open('./data/data.csv', 'a')
writer_train = csv.writer(f_train)
try:
    with open('./data/buzzfeed3.json') as data_file:    
        data = json.load(data_file)
        for i in xrange(len(data)):
            writer_train.writerow((data[i]["article_title"].encode('utf-8').replace(',', ' '), '1'))
finally:
    f_train.close()

In [13]:
reload(sys)
sys.setdefaultencoding("utf-8")
f_train = open('../flask_app/data/data.csv', 'a')
writer_train = csv.writer(f_train)

try:
    with open("../flask_app/data/new_bait") as tsvfile:
        for row in tsvfile:
            writer_train.writerow((row, '1'))
finally:
    f_train.close()

In [19]:
with open('../flask_app/data/new_bait') as oldfile, open('../flask_app/data/new_bait_n', 'w') as newfile:
    for line in oldfile:
        paragraph = "".join(map(lambda s: s.rstrip('\n'), line))
        newfile.write(paragraph + ',0\n')

In [8]:
reload(sys)
sys.setdefaultencoding("utf-8")
#f_train = open('../flask_app/data/data.csv', 'a')
bad_words = ['Timeline Photos,']

with open('../flask_app/data/data.csv') as oldfile, open('../flask_app/data/data_n.csv', 'w') as newfile:
    for line in oldfile:
        if not any(bad_word in line for bad_word in bad_words):
            newfile.write(line)


In [1]:
#### Begin Here

# Handling Data for keras 
from __future__ import print_function
import os
import sys
import numpy as np
np.random.seed(1337)

import pandas as pd
import random
from sklearn.utils import shuffle

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.utils import np_utils



MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

df = pd.read_csv('./data/data.csv')

# Removing NaNs (only 5 rows)
df = df[pd.notnull(df['title'])]

pos = df[df['label'] == 1]
neg = df[df['label'] == 0]

# We have more neg samples than pos, so even things out
neg_sample = neg.sample(len(pos))

# Merging pos and neg
data = pos.append(neg_sample, ignore_index=True)

# Get Train, Test and Valid data ready
X, Y = np.array(data['title']), np.array(data['label'])



Using TensorFlow backend.


In [2]:
BASE_DIR = './data/'
GLOVE_DIR = BASE_DIR + '/glove.6B/'

# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')

embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))


Indexing word vectors.
Found 400000 word vectors.


In [3]:
#Passing through tokenizer
texts = X.tolist()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 41021 unique tokens.


In [4]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = np_utils.to_categorical(np.asarray(Y))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (96058, 1000)
Shape of label tensor: (96058, 2)


In [5]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(0.1 * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [6]:
# prepare embedding matrix
nb_words = len(word_index)
embedding_matrix = np.zeros((nb_words + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(nb_words + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [7]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='float64')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(2, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])



In [8]:
# happy learning!
model.fit(x_train, y_train, validation_data=(x_val, y_val), nb_epoch=1, batch_size=128)

Train on 86453 samples, validate on 9605 samples
Epoch 1/1
86453/86453 [==============================] - 1203s - loss: 0.3497 - acc: 0.8432 - val_loss: 0.3224 - val_acc: 0.8636


In [13]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Convolution1D, GlobalMaxPooling1D

print('Build model...')
model1 = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model1.add(embedding_layer)

# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:
model1.add(Convolution1D(nb_filter=128,
                        filter_length=3,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1))
# we use max pooling:
model1.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model1.add(Dense(250))
model1.add(Dropout(0.2))
model1.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model1.add(Dense(1))
model1.add(Activation('sigmoid'))

model1.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model1.fit(x_train, y_train, validation_data=(x_val, y_val), nb_epoch=1, batch_size=512)

Build model...


Exception: Error when checking model target: expected activation_2 to have shape (None, 1) but got array with shape (86453, 2)

In [15]:
# How to load and use weights from a checkpoint
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import numpy
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# create model
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))
# load weights
model.load_weights("weights.best.hdf5")
# Compile model (required to make predictions)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Created model and loaded weights from file")
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# estimate accuracy on whole dataset using loaded weights
scores = model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, 1000)          0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 1000, 100)     0           input_4[0][0]                    
____________________________________________________________________________________________________
convolution1d_10 (Convolution1D) (None, 996, 128)      64128       embedding_1[3][0]                
____________________________________________________________________________________________________
maxpooling1d_10 (MaxPooling1D)   (None, 199, 128)      0           convolution1d_10[0][0]           
___________________________________________________________________________________________

In [27]:
lens = map(len, X)
max(lens)

172

[38,
 41,
 41,
 43,
 75,
 78,
 51,
 56,
 44,
 38,
 49,
 23,
 65,
 63,
 37,
 45,
 52,
 51,
 52,
 35,
 45,
 21,
 47,
 49,
 56,
 55,
 53,
 70,
 52,
 63,
 59,
 42,
 46,
 30,
 59,
 54,
 59,
 49,
 58,
 53,
 79,
 30,
 64,
 61,
 64,
 83,
 50,
 46,
 52,
 48,
 51,
 55,
 48,
 82,
 40,
 57,
 106,
 40,
 25,
 64,
 56,
 72,
 69,
 54,
 57,
 38,
 52,
 39,
 37,
 72,
 49,
 53,
 63,
 50,
 96,
 44,
 44,
 70,
 53,
 32,
 52,
 33,
 47,
 65,
 55,
 68,
 42,
 60,
 47,
 40,
 58,
 51,
 54,
 57,
 41,
 57,
 48,
 41,
 71,
 40,
 92,
 37,
 59,
 50,
 71,
 53,
 50,
 52,
 47,
 59,
 61,
 63,
 58,
 57,
 41,
 19,
 59,
 63,
 43,
 59,
 42,
 61,
 49,
 53,
 34,
 39,
 59,
 36,
 46,
 35,
 61,
 59,
 66,
 58,
 61,
 60,
 51,
 58,
 50,
 58,
 63,
 45,
 58,
 49,
 58,
 89,
 52,
 57,
 55,
 79,
 77,
 28,
 45,
 58,
 54,
 64,
 26,
 60,
 39,
 53,
 64,
 26,
 59,
 39,
 33,
 26,
 40,
 65,
 37,
 32,
 42,
 52,
 47,
 49,
 70,
 56,
 23,
 83,
 46,
 47,
 74,
 46,
 40,
 55,
 46,
 23,
 44,
 46,
 103,
 53,
 63,
 48,
 55,
 64,
 54,
 50,
 26,
 43,
 50,
 39

In [138]:
x_train.shape

(86453, 500)